In [53]:
from numpy import genfromtxt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import codecs
import numpy as np
import os
import shutil
import torch
import yaml
from datetime import datetime
from torch.utils.data.sampler import RandomSampler
from torch.autograd import Variable
EMBEDDING_SIZE = 300

In [2]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [3]:
train = data.iloc[0:1611]
test = data[1611:]
print(train.shape)
print(test.shape)

(1611, 27)
(378, 27)


In [4]:
trainheadlines = data.iloc[0:1611, 2:27].values
trainLabels = data.iloc[0:1611, 1].values
testheadlines = data.iloc[1611:, 2:27].values
testLabels = data.iloc[1611:, 1].values

print(trainheadlines.shape)
print(trainLabels[:10])

(1611, 25)
[0 1 0 0 1 1 0 0 1 1]


In [107]:
trainheadlines = []
trainLabels = []
trainRawheadlines = []
for row in range(0,len(train.index)):
    trainRawheadlines += [str(x) for x in train.iloc[row,2:27]]
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
    trainLabels.append(int(train.iloc[row,1]))

basicvectorizer = CountVectorizer(min_df=1)
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

testheadlines = []
testLabels = []
testRawheadlines = []
for row in range(0,len(test.index)):
    testRawheadlines += [str(x) for x in test.iloc[row,2:27]]
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    testLabels.append(test.iloc[row,1])

basictest = basicvectorizer.transform(testheadlines)

embeddings_index = {}

(1611, 31675)


In [ ]:
# print(trainheadlines[0])
# print(basictrain[0])
basicvectorizer.inverse_transform([0])
print(basicvectorizer.inverse_transform(basictrain[0]))
cx = scipy.sparse.coo_matrix(basictrain[0])

for i,j,v in zip(cx.row, cx.col, cx.data):
    print("(%d, %d), %s word: %s" % (i,j,v))
    

In [130]:
embeddings_index_300 = {}
def load_embeddings(BASE_DIR = 'C:/Users/rachu/Google Drive/UCI/q3/Independent Study'):
    with open('glove.6B.300d.txt', encoding = 'utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index_300[word] = coefs
    print('Found %s word vectors.' % len(embeddings_index_300))
load_embeddings()


Found 400000 word vectors.


In [132]:
print(embeddings_index_300['misdoings'])
embeddings_index['hi']-embeddings_index['hello']

[  2.92420000e-01  -3.32830012e-01  -2.96090007e-01  -2.37700008e-02
  -3.52970004e-01   7.01899976e-02   2.01959997e-01  -7.39279985e-02
  -1.68400005e-01   7.52699971e-01  -4.88150001e-01   2.17490003e-01
   2.49829993e-01  -1.37740001e-01  -3.59130017e-02  -4.74629998e-01
  -1.58480003e-01   5.40149994e-02   8.44030001e-04   2.77940005e-01
   2.51810014e-01  -5.38609996e-02  -2.23940000e-01  -4.16440010e-01
  -4.96379994e-02   3.01690012e-01  -4.63209987e-01   1.89480007e-01
  -4.67309989e-02  -4.99739982e-02   8.67350027e-02  -6.00639991e-02
   6.57599986e-01  -7.75470018e-01   5.81740022e-01  -3.19620013e-01
   1.94370002e-01   4.63160008e-01   4.78599995e-01  -3.43510002e-01
  -1.36140004e-01  -4.44689989e-01  -2.11750001e-01   9.98529978e-03
   5.43590002e-02  -4.73320000e-02  -1.36989996e-01   4.81440008e-01
   3.73030007e-01   3.37379992e-01   4.78810012e-01  -4.35350016e-02
   1.04889996e-01   2.36489996e-01  -1.73370004e-01   1.31650001e-01
   2.84110010e-01  -4.52429987e-02

KeyError: 'hi'

In [186]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
import math

class LogisticLayer(nn.Module):
    def __init__(self, input_dim):
        super(LogisticLayer, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, 2)
#         self.fc2 = nn.Linear(60, 2)

    def forward(self, headlines):
#         words = basicvectorizer.inverse_transform(headlines)
        embeddingSum = [float('-inf')]*self.input_dim
        for i, headline in enumerate(headlines):
            embeddingHeadline = [0]*self.input_dim
            words = basicvectorizer.build_tokenizer()(headline)
            for word in words:
                if word in embeddings_index_300:
                    embeddingHeadline += embeddings_index_300[word]
            embeddingSum = np.maximum(embeddingSum, embeddingHeadline)
        embeddingSumVar = Variable(torch.from_numpy(np.asarray(embeddingSum))).view(1, 1, -1).float().cuda()
        fcOutput = (self.fc1(embeddingSumVar))
        log_probs = F.log_softmax(fcOutput[0], dim=-1)
        return log_probs


In [187]:
# Logistic training

# logisticLayer = LogisticLayer(basictrain[0].shape[1]).cuda()
logisticLayer = LogisticLayer(EMBEDDING_SIZE).cuda()
learning_rate = 0.1
logisticOptimizer = torch.optim.Adam(logisticLayer.parameters(), lr=learning_rate)
epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()
cell = None
hidden = None
while epoch < 70:
    
    total_loss = 0
    for i in range(len(trainLabels)):
        logisticOptimizer.zero_grad()
        old_hidden = hidden
        log_probs = logisticLayer(trainRawheadlines[i*25:(i+1)*25])
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()        
    logisticOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 5) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)))
    epoch = epoch + 1

0 300 [ 1.02880001  0.71915001  0.46759     0.63142002  0.26254001  0.65047002
  0.34327     0.55716997  0.46055999 -0.15496001  0.62375998  0.35371
  0.27254     0.21006     0.41        1.00619996  0.39116001  0.2834
  0.36319     0.29064     0.46483999  0.53201997  1.1142      0.29918
  0.36074999  0.47420999  0.44369     0.49489999  0.2507      0.50234997
  0.46094999  0.39071     0.37990001  0.51413     0.99492002  0.28031999
  0.40974     0.40426001  0.23894     0.65875     0.41251001  0.70084
  0.23122001  0.30421999  0.79089999  0.20926     0.60563999  0.47692999
  0.65368998  0.44202     0.54155999  0.14662001  0.54491001  0.48897001
  0.30493999  0.45967999  0.33926001  0.83420998  0.32853001  0.89749998
  0.50041002  0.44286001  0.70697999  0.32464999  0.61337     0.87279999
  0.57744002  0.7349      0.67330003  0.46090001  0.62743002  0.26072001
  0.18256     0.28264999  0.53969997  0.78162998  0.54934001  0.80282003
  0.11527     0.20996     0.54557002  0.07099     0.43467 

16 300 [  1.24480002e-01   3.83100003e-01   1.43800005e-01  -6.31180033e-02
   1.08340003e-01   4.79189992e-01   5.15600026e-01   7.67090023e-01
   2.53369987e-01  -8.55449975e-01   3.66479993e-01   2.53490001e-01
   8.94649997e-02   5.68570018e-01   3.07720006e-01   1.38820007e-01
   1.58160001e-01  -5.41000022e-03   1.44700006e-01   2.65120000e-01
   2.07749993e-01   5.58600008e-01   6.18529975e-01   3.50989997e-01
  -1.28150001e-01   5.88339984e-01   2.36110002e-01   1.50240004e-01
   3.93190011e-02   4.64150012e-01   5.03310025e-01   8.35770011e-01
  -2.86940001e-02  -1.06769996e-02  -8.31450000e-02   1.65120006e-01
   6.16289973e-01   1.81240007e-01   1.07579994e+00   2.22929999e-01
   3.90949994e-01   1.65220007e-01   4.17860001e-01   2.66530007e-01
   5.59939981e-01   2.35919997e-01   3.06019992e-01   3.14190000e-01
   2.06479996e-01   2.71059990e-01   2.88010001e-01   4.50670021e-03
   2.47180000e-01   1.54170007e-01  -2.51400005e-02   5.85630000e-01
   1.89909995e-01   4.09570

   3.00770015e-01  -5.49720004e-02   4.43919986e-01   3.28990002e-03]
14 300 300
15 300 [  7.35149980e-02   6.74740016e-01   4.52820003e-01   4.27689999e-01
   3.86770010e-01   5.28320014e-01   3.02379996e-01   4.93770003e-01
   2.76659995e-01  -2.88029999e-01   7.22410023e-01  -3.55390012e-02
   2.89409995e-01  -2.65229996e-02   7.30120003e-01   3.93629998e-01
   6.77070022e-01   6.77129984e-01  -1.79760009e-02   5.29540002e-01
  -1.05540000e-01   9.06920016e-01   2.10120007e-01   1.11510003e+00
   4.40340012e-01   1.29529998e-01   3.53599995e-01   3.49689990e-01
   9.48690027e-02   6.56029999e-01   8.05850029e-01   6.79329991e-01
  -4.90790009e-02  -6.85890019e-02   6.15890026e-01   6.13749996e-02
   7.24150002e-01   1.91100001e-01   5.58600008e-01   4.90770012e-01
   1.19790003e-01   1.38469994e-01   3.48060012e-01   5.85439987e-02
  -1.28859997e-01   3.48989993e-01   6.19009972e-01   3.87230009e-01
   3.40449989e-01  -1.32090002e-01   2.54790008e-01   3.37769985e-02
  -4.07369994e-

9 300 [ 0.073515    0.25736001  0.2359      0.19299001 -0.29611     0.24525
  0.28196001  0.17337     0.46055999 -1.8477      0.41646001 -0.12955
  0.28941     0.0072891   0.26038     0.39363    -0.23193     0.065956
 -0.017976   -0.15155999 -0.10554     0.09882     0.21012001  0.49419999
  0.44034001  0.027245    0.11274     0.49489999 -0.042132    0.08476
  0.50555998  0.062284    0.20223001  0.038279   -0.69314998 -0.1214
  0.089846    0.29701999  0.55860001  0.24185    -0.13194001  0.13846999
  0.0025406   0.14981     0.25536001  0.31973001 -0.04452     0.38723001
  0.086896   -0.13209     0.28694001  0.013552   -0.040737    0.31749001
  0.067802    0.20353    -0.19889     0.60789001  0.013141    0.11997
  0.24637     0.43593001  0.30621001  0.086261    0.039448   -0.45570001
  0.21081001  0.25889     0.14666     0.30950001  0.43849     0.34619999
  0.28852999  0.103       0.32211     0.0085505   0.61328     0.32201999
  0.10784    -0.14917     0.12611     0.46024999 -0.060377    0

23 300 300
24 300 [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -in

...

  0.72907001  0.89411998  0.42302001  0.37086999 -0.019597    0.57343   ]
6 300 300
7 300 [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -in

24 300 [  2.94919997e-01   2.88910002e-01   2.58150011e-01   1.20169997e-01
   3.85780007e-01   3.46399993e-01   7.30859995e-01   2.00020000e-01
   2.80299991e-01  -1.42530000e+00   4.07029986e-01   5.13520002e-01
   7.22879991e-02   2.60490000e-01   2.81439990e-01   4.33239996e-01
   1.71649996e-02   2.24720001e-01   5.69069982e-01   1.81070000e-01
   3.54559988e-01   7.41119981e-01   6.18529975e-01   2.99180001e-01
   9.21249986e-02   1.04350001e-01   3.73290002e-01   8.39179978e-02
   1.65229999e-02   5.81210017e-01   2.80739993e-01   5.93999982e-01
   1.78509995e-01   4.43879992e-01  -6.77190006e-01   1.89060003e-01
   3.48980010e-01   1.81240007e-01   3.54880005e-01   3.21720004e-01
   5.67179978e-01   1.66490003e-01   2.91660011e-01   6.81890011e-01
   5.78170002e-01   3.61510009e-01   4.31010008e-01   5.29089987e-01
   4.13300004e-03   2.87739992e-01   1.57440007e-01   2.73519993e-01
   2.84830004e-01   3.70680004e-01   5.19110024e-01   5.63799977e-01
   1.09279998e-01   3.18260

  0.67093998  0.52478999  0.27314001  0.054062    0.34366     0.59886003]
9 300 300
10 300 [-0.081429    0.81962001  0.29508999  0.60456997  0.093779    0.37698999
  0.27623001  0.61967999  0.54325002 -0.75998002  0.27851999  0.55572999
  0.30677     0.24837001  0.11336     1.00539994  0.53446001  0.2297
  0.034859    0.77762997  0.52904999  0.3811      0.50549001  0.12646
  0.12111    -0.032627    0.40417999  0.19445001  0.49757999  0.59614998
  0.46123001  0.41152999  0.43200999  0.26078001  1.09249997 -0.14328
  0.33168     0.15638     0.032873    0.92047     0.71924001  0.82415998
 -0.072371    0.39847001  0.15030999  0.40349999  0.59078997  0.47246999
  0.31995001  0.22393    -0.078332    0.059203    0.18745001  0.69209999
  0.094913    0.11624    -0.013457    0.29385     0.36302999  0.29247001
  0.18599001  0.82350999  0.22400001  0.27355999  0.025575    0.57426
  0.65542001  0.38093001  0.51374     1.00960004  0.54575998  0.28628001
  0.30353999  0.45978001  0.61870998  0.320320

  0.32536     1.64950001  0.43000001  0.25485     0.38508001  0.61396003]
7 300 300
8 300 [ 0.56928998  0.13733999  0.73329002 -0.36636001  0.37512001  0.33194
  0.13601001  0.60337001  1.21060002 -0.18798999  0.48302001  0.81498998
  0.70168     0.21006     0.55057001  0.41628     0.69766003  0.58020002
  0.011473    0.18107     0.31347999  0.14933001  0.26076999  0.29918
  0.85219997 -0.036147    0.49680001  0.53788     0.71109998  0.58842999
  0.58187997  0.3371      0.45438999  0.44387999  0.42583999  0.1802
  0.64182001  0.18735     0.40437001  0.30526     0.46184999  0.0018967
  0.31494001  0.78850001  0.54993999 -0.098069    0.50986999  0.47692999
  0.099733    0.083287    0.2244      0.57818002  0.22688     0.65456998
  0.38863     0.55294001  0.25797001  0.51005     0.40371001 -0.18503
  0.4377      0.31494999  0.70697999  0.31733     0.45289001  0.15548
  0.54540998  0.26399001  0.24535     0.27434999  0.29815     0.40184
  0.16316     0.45849001  0.61896998  0.19444001  0.46

ValueError: operands could not be broadcast together with shapes (600,) (300,) 

In [152]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf

class LSTMLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LSTMLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lstmLayers = []
        for i in range(25):
            self.lstmLayers.append(nn.LSTM(input_dim, output_dim))
        self.basicLSTM = nn.LSTM(input_dim, output_dim)
        self.fc = nn.Linear(output_dim, 2)

    def forward(self, headlines, hidden = None, cell = None):
        outputs = []
        new_hiddens = []
        if hidden is None:
            hidden = Variable(torch.zeros(1, 1, self.output_dim)).float()
            cell = Variable(torch.zeros(1, 1, self.output_dim)).float()
        hidden = hidden.cuda()
        cell = cell.cuda()
        
        words = basicvectorizer.inverse_transform(headlines)
        embeddingSum = [0]*self.input_dim
        for word in words[0]:
            if word in embeddings_index:
                embeddingSum += embeddings_index[word]
        embeddingSumVar = Variable(torch.from_numpy(np.asarray(embeddingSum))).view(1, 1, -1).float().cuda()
#         print(embeddingSum)
        output, (hidden, cell) = self.basicLSTM(embeddingSumVar, (hidden, cell))
        log_probs = F.log_softmax(self.fc(output[0]), dim=-1)
        return log_probs, hidden, cell


In [227]:
# LSTM Training

lstmLayer = LSTMLayer(EMBEDDING_SIZE, 30).cuda()

learning_rate = 0.3
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=learning_rate)

epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()

while epoch < 5:
    cell = None
    hidden = None
    total_loss = 0
    for (i, headlines) in enumerate(basictrain):
        if i == 500:
            break
        lstmOptimizer.zero_grad()
        log_probs, hidden, cell = lstmLayer(headlines, hidden, cell)
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
        
    lstmOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 10) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)), end='\n')
    epoch = epoch + 1

2018-05-16 11:12:23.439718 :: Epoch 0 :: Iter 501 / 1611 :: train loss: 347.2811


In [76]:
state_dict = {
    'epoch': epoch,
    'logisticLayer': logisticLayer.state_dict(),
    'logisticOptimizer': logisticOptimizer.state_dict()
}
ckpt_path = 'data\ckpt\basic_most\model.pt'
# torch.save(state_dict, '/data/ckpt/basic_most/model.pt')
torch.save(state_dict, 'bow_log_model_full_epoch130_mindf_3_emb300.pt')

In [162]:
# del lstmLayer
# del lstmOptimizer
# del train_loss
lstmLayer = LSTMLayer(EMBEDDING_SIZE, 100).cuda()
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=0.3)
ckpt = torch.load('bow_log_model_full_epoch200_mindf_3_emb300.pt')
lstmLayer.load_state_dict(ckpt['lstmLayer'])
lstmOptimizer.load_state_dict(ckpt['lstm_optimizer'])

In [168]:
trainPredictions = []
log_probs = []
h = hidden
c = cell
for i in range(len(trainLabels)):
#     log_prob = logisticLayer(trainheadlines[i])
    log_prob = logisticLayer(trainRawheadlines[i*25:(i+1)*25])
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        trainPredictions.append(0)
    else:
        trainPredictions.append(1)

In [216]:
# lstmLayer.cpu()
trainPredictions = []
log_probs = []
h = hidden
c = cell
for i in range(len(trainLabels)):
    log_prob, h, c = lstmLayer(basictrain[i], h, c)
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        trainPredictions.append(0)
    else:
        trainPredictions.append(1)

In [169]:
print('Training Accuracy:', sum(np.asarray(trainPredictions) == np.asarray(trainLabels))/len(trainLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(trainLabels, trainPredictions))

Training Accuracy: 0.473618870267
[[731   7]
 [841  32]]


In [139]:

testPredictions = []
log_probs = []
h = None
c = None
for i in range(len(testLabels)):
#     log_prob = logisticLayer(testheadlines[i])
    log_prob = logisticLayer(testheadlines[i*25:(i+1)*25])
    log_probs.append(log_prob[0])
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        testPredictions.append(0)
    else:
        testPredictions.append(1)

In [ ]:

testPredictions = []
log_probs = []
h = None
c = None
for i in range(len(testLabels)):
    log_prob,h,c = lstmLayer(basictest[i],h,c)
    log_probs.append(log_prob[0])
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        testPredictions.append(0)
    else:
        testPredictions.append(1)

In [163]:
print('Test Accuracy:', sum(np.asarray(testPredictions) == np.asarray(testLabels))/len(testLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(testLabels, testPredictions))

Test Accuracy: 0.492063492063
[[178   8]
 [184   8]]
